In [ ]:
!pip -q install -U langgraph langchain-openai

import os, json, uuid, copy, math, re, operator
from typing import Any, Dict, List, Optional
from typing_extensions import TypedDict, Annotated

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import interrupt, Command

def _set_env_openai():
    if os.environ.get("OPENAI_API_KEY"):
        return
    try:
        from google.colab import userdata
        k = userdata.get("OPENAI_API_KEY")
        if k:
            os.environ["OPENAI_API_KEY"] = k
            return
    except Exception:
        pass
    import getpass
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI_API_KEY: ")

_set_env_openai()

MODEL = os.environ.get("OPENAI_MODEL", "gpt-4o-mini")
llm = ChatOpenAI(model=MODEL, temperature=0)

In [ ]:
SAMPLE_LEDGER = [
    {"txn_id": "T001", "name": "Asha", "email": "ASHA@Example.com", "amount": "1,250.50", "date": "12/01/2025", "note": "Membership renewal"},
    {"txn_id": "T002", "name": "Ravi", "email": "ravi@example.com", "amount": "-500", "date": "2025-12-02", "note": "Chargeback?"},
    {"txn_id": "T003", "name": "Sara", "email": "sara@example.com", "amount": "700", "date": "02-12-2025", "note": "Late fee waived"},
    {"txn_id": "T003", "name": "Sara", "email": "sara@example.com", "amount": "700", "date": "02-12-2025", "note": "Duplicate row"},
    {"txn_id": "T004", "name": "Lee", "email": "lee@example.com", "amount": "NaN", "date": "2025/12/03", "note": "Bad amount"},
]

ALLOWED_OPS = {"replace", "remove", "add"}

def _parse_amount(x):
    if isinstance(x, (int, float)):
        return float(x)
    if isinstance(x, str):
        try:
            return float(x.replace(",", ""))
        except:
            return None
    return None

def _iso_date(d):
    if not isinstance(d, str):
        return None
    d = d.replace("/", "-")
    p = d.split("-")
    if len(p) == 3 and len(p[0]) == 4:
        return d
    if len(p) == 3 and len(p[2]) == 4:
        return f"{p[2]}-{p[1]}-{p[0]}"
    return None

def profile_ledger(rows):
    seen, anomalies = {}, []
    for i, r in enumerate(rows):
        if _parse_amount(r.get("amount")) is None:
            anomalies.append(i)
        if r.get("txn_id") in seen:
            anomalies.append(i)
        seen[r.get("txn_id")] = i
    return {"rows": len(rows), "anomalies": anomalies}

def apply_patch(rows, patch):
    out = copy.deepcopy(rows)
    for op in sorted([p for p in patch if p["op"] == "remove"], key=lambda x: x["idx"], reverse=True):
        out.pop(op["idx"])
    for op in patch:
        if op["op"] in {"add", "replace"}:
            out[op["idx"]][op["field"]] = op["value"]
    return out

def validate(rows):
    issues = []
    for i, r in enumerate(rows):
        if _parse_amount(r.get("amount")) is None:
            issues.append(i)
        if _iso_date(r.get("date")) is None:
            issues.append(i)
    return {"ok": len(issues) == 0, "issues": issues}

In [ ]:
class TxnState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    raw_rows: List[Dict[str, Any]]
    sandbox_rows: List[Dict[str, Any]]
    patch: List[Dict[str, Any]]
    validation: Dict[str, Any]
    approved: Optional[bool]

def node_profile(state):
    p = profile_ledger(state["raw_rows"])
    return {"messages": [AIMessage(content=json.dumps(p))]}

def node_patch(state):
    sys = SystemMessage(content="Return a JSON patch list fixing amounts, dates, emails, duplicates")
    usr = HumanMessage(content=json.dumps(state["raw_rows"]))
    r = llm.invoke([sys, usr])
    patch = json.loads(re.search(r"\[.*\]", r.content, re.S).group())
    return {"patch": patch, "messages": [AIMessage(content=json.dumps(patch))]}

def node_apply(state):
    return {"sandbox_rows": apply_patch(state["raw_rows"], state["patch"])}

def node_validate(state):
    v = validate(state["sandbox_rows"])
    return {"validation": v, "messages": [AIMessage(content=json.dumps(v))]}

def node_approve(state):
    decision = interrupt({"validation": state["validation"]})
    return {"approved": decision == "approve"}

def node_commit(state):
    return {"messages": [AIMessage(content="COMMITTED")]}

def node_rollback(state):
    return {"messages": [AIMessage(content="ROLLED BACK")]}

In [ ]:
builder = StateGraph(TxnState)

builder.add_node("profile", node_profile)
builder.add_node("patch", node_patch)
builder.add_node("apply", node_apply)
builder.add_node("validate", node_validate)
builder.add_node("approve", node_approve)
builder.add_node("commit", node_commit)
builder.add_node("rollback", node_rollback)

builder.add_edge(START, "profile")
builder.add_edge("profile", "patch")
builder.add_edge("patch", "apply")
builder.add_edge("apply", "validate")

builder.add_conditional_edges(
    "validate",
    lambda s: "approve" if s["validation"]["ok"] else "rollback",
    {"approve": "approve", "rollback": "rollback"}
)

builder.add_conditional_edges(
    "approve",
    lambda s: "commit" if s["approved"] else "rollback",
    {"commit": "commit", "rollback": "rollback"}
)

builder.add_edge("commit", END)
builder.add_edge("rollback", END)

app = builder.compile(checkpointer=InMemorySaver())

In [4]:
def run():
    state = {
        "messages": [],
        "raw_rows": SAMPLE_LEDGER,
        "sandbox_rows": [],
        "patch": [],
        "validation": {},
        "approved": None,
    }

    cfg = {"configurable": {"thread_id": "txn-demo"}}
    out = app.invoke(state, config=cfg)

    if "__interrupt__" in out:
        print(json.dumps(out["__interrupt__"], indent=2))
        decision = input("approve / reject: ").strip()
        out = app.invoke(Command(resume=decision), config=cfg)

    print(out["messages"][-1].content)

run()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.8 MB/s eta 0:00:00
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	profile(profile)
	propose_patch(propose_patch)
	apply_patch(apply_patch)
	validate(validate)
	approve(approve)
	commit(commit)
	rollback(rollback)
	__end__([<p>__end__</p>]):::last
	__start__ --> profile;
	apply_patch --> validate;
	approve -.-> apply_patch;
	approve -.-> commit;
	approve -.-> rollback;
	profile --> propose_patch;
	propose_patch --> apply_patch;
	validate -.-> approve;
	validate -.-> propose_patch;
	validate -.-> rollback;
	commit --> __end__;
	rollback --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill